In [331]:
import pandas as pd
from functools import reduce

In [332]:
# Read in school
school = pd.read_csv("raw_data/schools_complete.csv")
school = school.rename(columns={"name": "school"})
school.set_index("school", inplace = True)

In [333]:
# Read in students
stud = pd.read_csv("raw_data/students_complete.csv")

In [334]:
# DISTRICT SUMMARY
# Assuming passing score is 70% (not specified in HW instructions)

totSch = len(school.index)
totStu = len(stud.index)
totBudg = school["budget"].sum()
avgMath = round(stud["math_score"].mean(),2)
avgRead = round(stud["reading_score"].mean(),2)
passMath = round((stud["math_score"][stud["math_score"] >= 70].count()) / totStu * 100,2)
passRead = round((stud["reading_score"][stud["reading_score"] >= 70].count()) / totStu * 100,2)
passAll = round((float(passMath) + passRead) / 2, 2)

district = pd.DataFrame({
          "Total Schools": [totSch],
          "Total Students": ["{:,}".format(totStu)],
          "Total Budget": ["$" + "{:,}".format(totBudg)],
          "Average Math Score": [avgMath],
          "Average Reading Score": [avgRead],
          "% Passing Math": [passMath],
          "% Passing Reading": [passRead],
          "Overall Passing Rate": [passAll],
          })

district[["% Passing Math", "% Passing Reading", "Overall Passing Rate"]] = district[["% Passing Math", "% Passing Reading", "Overall Passing Rate"]].applymap("{:,.2f}".format) + "%"

# Set order of columns

district = district[["Total Schools","Total Students","Total Budget","Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","Overall Passing Rate"]]

In [335]:
# ANSWER - DISTRICT SUMMARY

district

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,15,"39,170","$24,649,428",78.99,81.88,74.98%,85.81%,80.40%


In [336]:
# SCHOOL SUMMARY - calc scores

studSchool = stud.groupby("school")
avgMath_School = round(studSchool["math_score"].mean(),2).to_frame()
avgRead_School = round(studSchool["reading_score"].mean(),2).to_frame()
students_School = studSchool["school"].count().to_frame()              # Number of students in school
passMathn = studSchool["math_score"].agg({lambda x: (x >= 70).sum()})  # Number of students who passed math
passReadn = studSchool["reading_score"].agg({lambda x: (x >= 70).sum()})   # Number of students who passed reading


In [337]:
# SCHOOL SUMMARY - create df of scores

from functools import reduce

mergeVars = [avgMath_School, avgRead_School, passMathn, passReadn, students_School]
scores_School = reduce(lambda left, right: pd.merge(left, right, left_index = True, right_index = True), mergeVars)
scores_School.columns = ["Average Math Score", "Average Reading Score", "n Passed Math", "n Passed Reading", "# Students"]
scores_School["% Passing Math"] = round(scores_School["n Passed Math"] / scores_School["# Students"] * 100,2)
scores_School["% Passing Reading"] = round(scores_School["n Passed Reading"] / scores_School["# Students"] * 100,2)
scores_School["Overall Passing Rate"] = round((scores_School["% Passing Math"]+scores_School["% Passing Reading"])/2, 2)
scores_School = scores_School[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "Overall Passing Rate"]]

In [338]:
# SCHOOL SUMMARY - merge dfs

schoolSummary = school.merge(scores_School, left_index= True, right_index= True)
schoolSummary.sort_index(inplace = True)            # Sort schools alphabetically

# Format columns
schoolSummary["Total Students"] = schoolSummary["size"].map("{:,}".format)    
schoolSummary["Total Budget"] = "$" + schoolSummary["budget"].map("{:,}".format)
schoolSummary["Per Student Budget"] = "$" + (schoolSummary["budget"]/schoolSummary["size"]).map("{:,.2f}".format)
schoolSummary = schoolSummary.drop(["School ID", "size", "budget"], axis = 1)
schoolSummary = schoolSummary.rename(columns = {"type": "School Type"})

# Set order of columns
schoolSummary = schoolSummary[['School Type',
 'Total Students',
 'Total Budget',
 'Per Student Budget',
 'Average Math Score',
 'Average Reading Score',
 '% Passing Math',
 '% Passing Reading',
 'Overall Passing Rate']]

In [339]:
# ANSWER - SCHOOL SUMMARY

schoolSummary_answer = schoolSummary.reset_index()
schoolSummary_answer[["% Passing Math", "% Passing Reading", "Overall Passing Rate"]] = schoolSummary_answer[["% Passing Math", "% Passing Reading", "Overall Passing Rate"]].applymap("{:,.2f}".format) + "%"
schoolSummary_answer

,school,School Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Bailey High School,District,"4,976","$3,124,928",$628.00,77.05,81.03,66.68%,81.93%,74.31%
1,Cabrera High School,Charter,"1,858","$1,081,356",$582.00,83.06,83.98,94.13%,97.04%,95.58%
2,Figueroa High School,District,"2,949","$1,884,411",$639.00,76.71,81.16,65.99%,80.74%,73.36%
3,Ford High School,District,"2,739","$1,763,916",$644.00,77.10,80.75,68.31%,79.30%,73.81%
4,Griffin High School,Charter,"1,468","$917,500",$625.00,83.35,83.82,93.39%,97.14%,95.26%
5,Hernandez High School,District,"4,635","$3,022,020",$652.00,77.29,80.93,66.75%,80.86%,73.81%
6,Holden High School,Charter,427,"$248,087",$581.00,83.80,83.81,92.51%,96.25%,94.38%
7,Huang High School,District,"2,917","$1,910,635",$655.00,76.63,81.18,65.68%,81.32%,73.50%
8,Johnson High School,District,"4,761","$3,094,650",$650.00,77.07,80.97,66.06%,81.22%,73.64%
9,Pena High School,Charter,962,"$585,858",$609.00,83.84,84.04,94.59%,95.95%,95.27%


In [340]:
# ANSWER - Top Performing Schools (By Passing Rate)

topSchools = schoolSummary.sort_values("Overall Passing Rate", ascending = False).reset_index()
topSchools = topSchools.loc[0:4,:]
topSchools[["% Passing Math", "% Passing Reading", "Overall Passing Rate"]] = topSchools[["% Passing Math", "% Passing Reading", "Overall Passing Rate"]].applymap("{:,.2f}".format) + "%"
topSchools

,school,School Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Cabrera High School,Charter,"1,858","$1,081,356",$582.00,83.06,83.98,94.13%,97.04%,95.58%
1,Thomas High School,Charter,"1,635","$1,043,130",$638.00,83.42,83.85,93.27%,97.31%,95.29%
2,Pena High School,Charter,962,"$585,858",$609.00,83.84,84.04,94.59%,95.95%,95.27%
3,Griffin High School,Charter,"1,468","$917,500",$625.00,83.35,83.82,93.39%,97.14%,95.26%
4,Wilson High School,Charter,"2,283","$1,319,574",$578.00,83.27,83.99,93.87%,96.54%,95.21%


In [341]:
# ANSWER - Bottom Performing Schools (By Passing Rate)

lowSchools = schoolSummary.sort_values("Overall Passing Rate", ascending = True).reset_index()
lowSchools = lowSchools.loc[0:4,:]
lowSchools[["% Passing Math", "% Passing Reading", "Overall Passing Rate"]] = lowSchools[["% Passing Math", "% Passing Reading", "Overall Passing Rate"]].applymap("{:,.2f}".format) + "%"
lowSchools

,school,School Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Rodriguez High School,District,"3,999","$2,547,363",$637.00,76.84,80.74,66.37%,80.22%,73.30%
1,Figueroa High School,District,"2,949","$1,884,411",$639.00,76.71,81.16,65.99%,80.74%,73.36%
2,Huang High School,District,"2,917","$1,910,635",$655.00,76.63,81.18,65.68%,81.32%,73.50%
3,Johnson High School,District,"4,761","$3,094,650",$650.00,77.07,80.97,66.06%,81.22%,73.64%
4,Ford High School,District,"2,739","$1,763,916",$644.00,77.10,80.75,68.31%,79.30%,73.81%


In [342]:
# Math Scores by Grades

#9th grade

math9 = round(stud[["school","math_score"]][stud["grade"] == "9th"]  \
       .groupby(["school"])["math_score"].mean(),2).to_frame()
math9 = math9.rename(columns = {"math_score": "9th"})

#10th grade

math10 = round(stud[["school","math_score"]][stud["grade"] == "10th"]  \
       .groupby(["school"])["math_score"].mean(),2).to_frame()
math10 = math10.rename(columns = {"math_score": "10th"})

#11th grade

math11 = round(stud[["school","math_score"]][stud["grade"] == "11th"]  \
       .groupby(["school"])["math_score"].mean(),2).to_frame()
math11 = math11.rename(columns = {"math_score": "11th"})

#12th grade

math12 = round(stud[["school","math_score"]][stud["grade"] == "12th"]  \
       .groupby(["school"])["math_score"].mean(),2).to_frame()
math12 = math12.rename(columns = {"math_score": "12th"})


# Merge grade dfs together

# ANSWER - Math Scores by Grade

mergemath = [math9, math10, math11, math12]
math_Grades = reduce(lambda left, right: pd.merge(left, right, left_index = True, right_index = True), mergemath)
math_Grades.reset_index()

,school,9th,10th,11th,12th
0,Bailey High School,77.08,77.00,77.52,76.49
1,Cabrera High School,83.09,83.15,82.77,83.28
2,Figueroa High School,76.40,76.54,76.88,77.15
3,Ford High School,77.36,77.67,76.92,76.18
4,Griffin High School,82.04,84.23,83.84,83.36
5,Hernandez High School,77.44,77.34,77.14,77.19
6,Holden High School,83.79,83.43,85.00,82.86
7,Huang High School,77.03,75.91,76.45,77.23
8,Johnson High School,77.19,76.69,77.49,76.86
9,Pena High School,83.63,83.37,84.33,84.12


In [343]:
# Reading Scores by Grades

#9th grade

read9 = round(stud[["school","reading_score"]][stud["grade"] == "9th"]  \
       .groupby(["school"])["reading_score"].mean(),2).to_frame()
read9 = read9.rename(columns = {"reading_score": "9th"})

#10th grade

read10 = round(stud[["school","reading_score"]][stud["grade"] == "10th"]  \
       .groupby(["school"])["reading_score"].mean(),2).to_frame()
read10 = read10.rename(columns = {"reading_score": "10th"})

#11th grade

read11 = round(stud[["school","reading_score"]][stud["grade"] == "11th"]  \
       .groupby(["school"])["reading_score"].mean(),2).to_frame()
read11 = read11.rename(columns = {"reading_score": "11th"})

#12th grade

read12 = round(stud[["school","reading_score"]][stud["grade"] == "12th"]  \
       .groupby(["school"])["reading_score"].mean(),2).to_frame()
read12 = read12.rename(columns = {"reading_score": "12th"})


# Merge grade dfs together 

# ANSWER - Reading Scores by Grade

mergeReading = [read9, read10, read11, read12]
reading_Grades = reduce(lambda left, right: pd.merge(left, right, left_index = True, right_index = True), mergeReading)
reading_Grades.reset_index()

,school,9th,10th,11th,12th
0,Bailey High School,81.30,80.91,80.95,80.91
1,Cabrera High School,83.68,84.25,83.79,84.29
2,Figueroa High School,81.20,81.41,80.64,81.38
3,Ford High School,80.63,81.26,80.40,80.66
4,Griffin High School,83.37,83.71,84.29,84.01
5,Hernandez High School,80.87,80.66,81.40,80.86
6,Holden High School,83.68,83.32,83.82,84.70
7,Huang High School,81.29,81.51,81.42,80.31
8,Johnson High School,81.26,80.77,80.62,81.23
9,Pena High School,83.81,83.61,84.34,84.59


In [344]:
# SCORES BY SCHOOL SPENDING

# Create df

schoolBudg = schoolSummary[['Per Student Budget',
                         'Average Math Score',
                         'Average Reading Score',
                         '% Passing Math',
                         '% Passing Reading',
                         'Overall Passing Rate']]       \
            .reset_index()

schoolBudg['Per Student Budget'] = pd.to_numeric(schoolBudg['Per Student Budget'].str.replace("$", ""))
schoolBudg = schoolBudg.drop("school", 1)

                                          
# Find school spending quartiles 

import numpy as np

schoolBudg["Per Student Budget"].quantile(q = [.25,.5,.75])

conditions = [
    (schoolBudg["Per Student Budget"] < 591.5),
    (schoolBudg["Per Student Budget"] >= 591.5) & (schoolBudg["Per Student Budget"] < 628.0),
    (schoolBudg["Per Student Budget"] >= 628.0) & (schoolBudg["Per Student Budget"] < 641.5),
    (schoolBudg["Per Student Budget"] >= 641.5)]

quartiles = ["Q1: < $591.5",
             "Q2: $591.5-628.0",
             "Q3: $628.0-641.5",
             "Q4: >= 641.5"]

schoolBudg["Spending Quartile"] = np.select(conditions, quartiles)

scores_bySpending = schoolBudg.groupby(["Spending Quartile"], as_index = False).mean().round(2)

# Format cols

scores_bySpending["Per Student Budget"] = "$" + scores_bySpending["Per Student Budget"].map("{:,.2f}".format)
scores_bySpending[["% Passing Math", "% Passing Reading", "Overall Passing Rate"]] = scores_bySpending[["% Passing Math", "% Passing Reading", "Overall Passing Rate"]].applymap("{:,.2f}".format) + "%"

In [345]:
## ANSWER - Scores by School Spending

scores_bySpending

,Spending Quartile,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Q1: < $591.5,$581.00,83.45,83.94,93.46%,96.61%,95.04%
1,Q2: $591.5-628.0,$611.33,83.52,83.86,93.95%,96.31%,95.13%
2,Q3: $628.0-641.5,$635.50,78.50,81.69,73.08%,85.05%,79.07%
3,Q4: >= 641.5,$650.25,77.02,80.96,66.70%,80.68%,73.69%


In [346]:
# SCORES BY SCHOOL SIZE

# Create df

schoolSize = schoolSummary[['Total Students',
                         'Average Math Score',
                         'Average Reading Score',
                         '% Passing Math',
                         '% Passing Reading',
                         'Overall Passing Rate']]       \
            .reset_index()

schoolSize = schoolSize.drop("school", 1)

schoolSize["Total Students"] = pd.to_numeric(schoolSize["Total Students"].str.replace(",", ""))

                                          
# Find school sizes 

import numpy as np

schoolSize["Total Students"].quantile(q = [.25,.5,.75])

conditions = [
    (schoolSize["Total Students"] < 1698),
    (schoolSize["Total Students"] >= 1698) & (schoolSize["Total Students"] < 3474),
    (schoolSize["Total Students"] >= 3474)]
sizes = ["Small: < 1,698", "Medium: 1,698-3,474", "Large: > 3,474"]

schoolSize["Size"] = np.select(conditions, sizes)


scores_bySize = schoolSize.groupby(["Size"], as_index = False).mean() \
                .round(2)                         \
                .sort_values(["Total Students"])    \
                .drop(["Total Students"], 1)

# # Format cols

scores_bySize[["% Passing Math", "% Passing Reading", "Overall Passing Rate"]] = scores_bySize[["% Passing Math", "% Passing Reading", "Overall Passing Rate"]].applymap("{:,.2f}".format) + "%"

In [347]:
# ANSWER - Scores by School Size

scores_bySize

,Size,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
2,"Small: < 1,698",83.60,83.88,93.44%,96.66%,95.05%
1,"Medium: 1,698-3,474",80.54,82.68,82.17%,89.63%,85.90%
0,"Large: > 3,474",77.06,80.92,66.46%,81.06%,73.76%


In [348]:
# SCORES BY SCHOOL TYPE

# Create df

schoolType = schoolSummary[['School Type',
                         'Average Math Score',
                         'Average Reading Score',
                         '% Passing Math',
                         '% Passing Reading',
                         'Overall Passing Rate']]       \
            .reset_index()

schoolType = schoolType.drop("school", 1)

scores_byType = schoolType.groupby(["School Type"], as_index = False).mean().round(2)
                                          
# Format cols

scores_byType[["% Passing Math", "% Passing Reading", "Overall Passing Rate"]] = scores_byType[["% Passing Math", "% Passing Reading", "Overall Passing Rate"]].applymap("{:,.2f}".format) + "%"

In [349]:
# ANSWER - Scores by School Type

scores_byType

,School Type,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Charter,83.47,83.90,93.62%,96.59%,95.10%
1,District,76.96,80.97,66.55%,80.80%,73.68%
